# 言語モデル

*Language Model*

単語の並びに確率を割り当てるモデル。単語の並びとは、要は文章のこと。文章生成に多く使われるので文章生成モデルと見てもいいが、当然別の用途もある。

最近では、かなり人間に近い文章を生成できるすげーモデルがLLM (Large Language Model)と称して一世を風靡している。

In [1]:
import random
from typing import List


---

## 最も簡単な言語モデル

まずは最も簡単な言語モデルを見てみよう。

言語モデルとは単語の並びに対して確率を割り当てるモデルであった。この確率を割り当てるアルゴリズムを出来る限りシンプルにしたモデルを考える。

具体的にどうするかというと、ランダムにする。いかなる単語の並びに対してランダムな確率を割り当てるモデルということ。実際に作ってみよう。

In [2]:
# 単語列を入力として、確率を出力する関数
def langage_model(sentence):
    prob = random.random() # 0 ~ 1 のランダムな値
    return prob

できた。実際に使ってみよう。適当な単語列を入力してみる。

In [3]:
sentence = "明日は晴れるといいな" # 適当な単語列 (文章)
prob = langage_model(sentence)
prob

0.8550537734635512

ランダムな確率が出力された。これが最も簡単な言語モデルである。


---

## 言語モデルを用いた文章生成

任意の言語モデルを用いて文章を生成する方法を見ていく。

前節では入力された単語列に対してなんらかの確率を割り当てる「言語モデル」と呼ばれるモデルを学んだ。では、このモデルを用いて文章を生成するにはどうすればいいだろう。答えは簡単。確率が高い単語の並びを選択するだけ。

モデルは「語彙」を持っており、その語彙から作れる全ての単語列に対して確率を割り当て、最も高い単語列を出力する。その単語列が、モデルが生成した文章となる。

では作っていこう、といきたいところだが、この生成手法には色々と問題がある。

今、モデルが持っている語彙から作れる全ての単語列に確率を割り当てると説明した。しかし、単語数に制限を掛けない場合、ある語彙から作れる単語列の数は無限である。また仮に単語数を制限したとしても、語彙の数が大きい場合は膨大な量になる。

そこで、**与えられた文脈**と持っている語彙から作れる単語列で考える。

例えば、モデルが以下の語彙を持っているとする。

- 今日
- 天気
- おいしい
- は
- いい

ここで、「今日 は いい」という単語列があったとする。このとき、この単語列と語彙から作れる文章を以下とする。

- 今日 は いい 今日
- 今日 は いい 天気
- 今日 は いい おいしい
- 今日 は いい は
- 今日 は いい いい

単語列の末尾に単語を1つ追加しただけだ。

これらに対して言語モデルで確率を割り当て、最も確率の高い単語列を出力する。そして、出力された単語列を再度モデルに与え、同様に単語列を生成する。これを繰り返すことで文章を生成する。これが、言語モデルを用いた文章生成である。

以上を踏まえると、文章生成における言語モデルの役割は「次の単語の予測」とも見て取れる。（文章生成においては、）与えられた文脈に続く単語を予測するモデルが言語モデルとなる。言語モデルが予測した単語を文章に追加し、その文章を再度言語モデルに与えて次の単語を予測し...と繰り返すことで文章を生成する。このような処理は**再帰的**な処理と呼ぶ。

なお、厳密には、言語モデルが予測するものは次の単語ではなくその確率分布である。次の単語は確率分布からのサンプリングによって得る。言語モデルは以下のように記述できる。

$$
p(w_t|w_1, w_2, \dots, w_{t-1})
$$

ある時刻$t$における単語$w_t$の確率分布$p(w_t)$を$t$より前の単語列$w_1, w_2, \dots, w_{t-1}$から決定する。

余談

確率分布からのサンプリングによって得られる出力は確率的なものである。一方で、「確率が最も高い単語を出力する」とすれば得られる出力は確定的なものとなる。どちらを採用するかは自由であるが、もしそのモデルを「生成モデル」と謳いたいのであれば確率的な出力を採用するべきだろう。

そもそも生成モデルとはデータが生まれるまでの確率的な過程を記述したものである。多くの生成モデルはどこかに乱数を用いた確率的な過程を含む。実際にChatGPTやStable Diffusionに全く同じプロンプトを与えても異なる文章・画像が生成されるだろう。

言語モデルの様な再帰的にデータを生成するモデルは**自己回帰モデル**（AR, Auto Regressive Model）と呼ばれる。文章の様な時系列データに多く用いられるが、画像の様な時系列でないデータにも活用できる。

では、この手法に基づいた言語モデルを作っていこう。

与えられた文脈から次の単語の確率分布を予測し、それに基づいて次の単語をサンプリングするモデルを作る。確率分布はランダムとする。

In [4]:
from typing import List

class LanguageModel:
    def __init__(self, vocab: List[str]):
        self.vocab = vocab # 語彙
        self.n_vocab = len(vocab) # 語彙数

    def dist_predict(self, sentence): # 単語列=文章を入力として、確率分布を出力する
        weights = [random.random() for _ in range(self.n_vocab)]
            # 0 ~ 1 のランダムな値を語彙数分生成
        dist = [weight / sum(weights) for weight in weights]
            # 正規化（確率分布に変換）
        return dist

    def sampling(self, dist): # 確率分布に従って単語をサンプリングする
        word, = random.choices(self.vocab, weights=dist)
        return word

    def __call__(self, context): # 文脈を受け取って次の単語を出力する
        dist = self.dist_predict(context) # 文脈を入力して確率分布を得る
        next_word = self.sampling(dist) # 単語をサンプリング
        return next_word


できた。

適当に語彙と文脈を与えて次の単語を出力させてみよう。

In [5]:
vocab = ["今日", "天気", "おいしい", "は", "いい"] # 語彙
model = LanguageModel(vocab) # 言語モデル

context = "今日はいい" # 文脈
word = model(context) # 次の単語をサンプリング
word

'おいしい'

この単語を文脈に追加して、再度次の単語を出力させる。

In [6]:
context = "今日はいい" + word # 得られた単語を文脈に追加
print(context) # 文脈
word = model(context) # 次の単語をサンプリング
word # 次の単語

今日はいいおいしい


'いい'

これを繰り返せば文章が出来そうだ。やってみよう。

In [7]:
n_words = 10 # 生成する単語数

for _ in range(n_words):
    word = model(context) # 次の単語を予測
    context += word # 文脈に単語を追加

context

'今日はいいおいしいおいしい天気はおいしいおいしいはおいしいおいしい今日いい'

出来た。これが言語モデルによる文章生成である。

今回は、いかなる文脈に対してランダムな確率分布を出力するモデルを作り、それを用いて文章を生成した。出力される確率分布がランダムなので、当然、めちゃくちゃな文章が生成される。

ただ、もしモデルが、我々が自然だと感じる単語の並びに高い確率を割り当てられるようになった場合はどうだろう。我々にとって自然な文章が生成できるようになるのではないだろうか。

次章以降では、人間が書いた大量の文章を用いて（学習させて）、モデルが自然な文章を生成できるようにする。自然な単語列に高い確率を割り当てられるようにする。